In [ ]:
import numpy as np
import tensorflow as tf
from pinn import get_network
from pinn.utils import pi_named, atomic_dress, connect_dist_grad
from pinn.models.base import export_model, get_train_op, MetricsCollector
import os, warnings
import tensorflow as tf
from glob import glob
from ase.collections import g2
from pinn.io import load_qm9, sparse_batch
from pinn import get_model, get_calc
from pinn.models.potential import make_metrics
from pinn.optimizers import get

In [ ]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = lambda: load_qm9(filelist, splits={'train':8, 'test':2}) #lambda is being used here to assign execution of load_qm9 to the variable "dataset"
train = lambda: dataset()['train'].repeat().shuffle(1000).apply(sparse_batch(256))
test = lambda: dataset()['test'].repeat().apply(sparse_batch(256))

In [ ]:
newtrain = dataset()['train'].shuffle(1000).apply(sparse_batch(256))
trainbatchcount = 0
for i in newtrain:
    trainbatchcount += 1 

print(trainbatchcount) #so 419 batches?

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-20 17:42:59.930520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-20 17:42:59.930631: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-20 17:43:00.027939: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


419


In [ ]:
newtest = dataset()['test'].shuffle(1000).apply(sparse_batch(100))
testbatchcount = 0
for i in newtest:
    testbatchcount += 1 

print(testbatchcount)

268


In [ ]:
params = {'optimizer': {'class_name': 'Adam', 'config': {'learning_rate': {'class_name': 'ExponentialDecay', 'config': {'initial_learning_rate': 0.0003, 'decay_steps': 10000, 'decay_rate': 0.994}}, 'clipnorm': 0.01}}, 'model_dir': '/Users/miguelnavaharris/Project/miguelmodels/newtrainloop', 'network': {'name': 'PiNet', 'params': {'depth': 4, 'rc': 4.0, 'atom_types': [1, 6, 7, 8, 9]}}, 'model': {'name': 'potential_model', 'params': {'learning_rate': 0.001}}}

In [ ]:
default_params = {
    ### Scaling and units # The loss function will be MSE((pred - label) * scale)
    # For vector/tensor predictions
    # the error will be pre-component instead of per-atom
    # e_unit is the unit of energy to report w.r.t the input labels
    # no f_unit yet, f_unit is just e_unit/input coordinate unit
    # e.g. if one have input in Hartree, scales it by 100 for training
    #      and output eV when report error
    #      then e_scale should be 100, and e_unit = hartree2evp
    'e_dress': {},  # element-specific energy dress
    'e_scale': 1.0,  # energy scale for prediction
    'e_unit': 1.0,  # output unit of energy during prediction
    # Loss function options
    ## Energy
    'max_energy': False,        # if set to float, omit energies larger than it
    'use_e_per_atom': False,    # use e_per_atom to calculate e_loss
    'log_e_per_atom': False,    # log e_per_atom and its distribution
                                # ^- this is forcely done if use_e_per_atom
    'use_e_weight': False,      # scales the loss according to e_weight
    ## Force
    'use_force': False,         # include force in loss function
    'max_force_comp': False,    # if set to float, omit forces components larger than it
    'no_force_comp': False,     # if set to int, use as maximum number of force components for a update
    'use_f_weight': False,      # scales the loss according to f_weights
    ## Stress
    'use_stress': False,        # include stress in Loss function
    ## L2
    'use_l2': False,
    ## Loss function multipliers
    'e_loss_multiplier': 1.0,
    'f_loss_multiplier': 1.0,
    's_loss_multiplier': 1.0,
    'l2_loss_multiplier': 1.0,
    'separate_errors': False,   # workaround at this point
}

In [ ]:
network = get_network(params['network'])
print(network)

In [ ]:
model_params = default_params.copy()
model_params.update(params['model']['params'])

In [ ]:
for batch in newtrain:
    batch = network.preprocess(batch)
    connect_dist_grad(batch)

In [ ]:
signature_dict = {'elems': tf.TensorSpec(shape=[], dtype=tf.float32),
                 'coord': tf.TensorSpec(shape=[], dtype=tf.float32),
                 'e_data': tf.TensorSpec(shape=[], dtype=tf.float32),
                 'ind_1': tf.TensorSpec(shape=[], dtype=tf.float32)}
#@tf.function can take this as argument like so:
#@tf.function(input_signature=([signature_dict])

In [ ]:
@tf.function
def train_loop(newtrain):

    optimizer = get(params['optimizer'])


    epochs = 1
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.

        for step, batch in enumerate(newtrain):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
            with tf.GradientTape() as tape:

                # Run the forward pass of the layer.
                # The operations that the layer applies
                # to its inputs are going to be recorded
                # on the GradientTape.
                pred = network(batch, training=True)  # Logits for this minibatch

                ind = batch['ind_1']
                nbatch = tf.reduce_max(ind)+1
                pred = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)

                metrics = make_metrics(batch, pred, model_params, tf.estimator.ModeKeys.TRAIN)

                # Compute the loss value for this minibatch.
                loss_value = tf.reduce_sum(metrics.LOSS)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = tape.gradient(loss_value, network.trainable_weights)

            # Run one step of gradient descent by updating
            # the value of the variables to minimize the loss.
            optimizer.apply_gradients(zip(grads, network.trainable_weights))

            if step % 100 == 0:
                print(
                    f"Training loss (for one batch) at step {step}: {float(loss_value)}"
                )
                print('100 steps')

In [ ]:
train_loop(newtrain)


Start of epoch 0


/Users/miguelnavaharris/miniforge3/envs/tffunction/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("concat_1:0", shape=(None,), dtype=int32), values=Tensor("concat:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/pi_net/gc_block_3/pi_layer_3/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/miguelnavaharris/miniforge3/envs/tffunction/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("concat_3:0", shape=(None,), dtype=int32), values=Tensor("concat_2:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/pi_net/gc_block_2/pi_layer_2/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.

Training loss (for one batch) at step Tensor("args_1:0", shape=(), dtype=int64): Tensor("Sum:0", shape=(), dtype=float32)
100 steps

Start of epoch 0


ValueError: in user code:

    File "/var/folders/2r/53clw21s5zv7ml4w3_xhd8v40000gq/T/ipykernel_99516/3938954701.py", line 40, in train_loop  *
        optimizer.apply_gradients(zip(grads, network.trainable_weights))
    File "/Users/miguelnavaharris/miniforge3/envs/tffunction/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 639, in apply_gradients  **
        self._create_all_weights(var_list)
    File "/Users/miguelnavaharris/miniforge3/envs/tffunction/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 823, in _create_all_weights
        _ = self.iterations
    File "/Users/miguelnavaharris/miniforge3/envs/tffunction/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 830, in __getattribute__
        return super(OptimizerV2, self).__getattribute__(name)
    File "/Users/miguelnavaharris/miniforge3/envs/tffunction/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 985, in iterations
        self._iterations 